In [1]:
print("223")

223


In [9]:
import os 
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
MODEL = "llama2"

In [29]:
from langchain_community.llms import Ollama
#from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_openai.embeddings import OpenAIEmbeddings

#if MODEL.startswith("gpt"):
#    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
#    embeddings = OpenAIEmbeddings()
#else:
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

#model.invoke("how to impress my boyfriend who likes business and build wealth and family ?")

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me the joke")

"Sure, here's a short one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Arun_Resume.pdf")
pages = loader.load_and_split()
pages


[Document(page_content='Arun Raja Manjini  \narunraja08@gmail.com |  (361) 228 -0045 | Hillsboro, OR  | linkedin.com/in/arunrjm/  \n \nOBJECTIVE  \nAs a n Engineer  my goal is to immerse myself in cutting -edge technologies, methodologies, and tools \nemployed in  pre-silicon & post -silicon validation, thereby broadening my expertise,  and learning latest \nvalidation methodologies /tools , and becoming an integral part of a team delivering high -quality  product.  \n \nEXPERIENCE  \nSystem Validation Engineer , Intel Corporation, USA                              Aug 2021 – Present  \n▪ Contributed to the test plans to functionally validate Intel Scalable coherent fabric’s features routing \nand cache coherency in LLC and other IO devices.  \n▪ Collaborated with the IP design Architecture / SoC  Emulation teams to review and debug of functional \nscenarios to functional/ performance validation.  \n▪ Implemented/ develop ed Python scripts for debug script and individual test cases, int

In [16]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [19]:
chain = prompt| model | parser  
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [25]:
chain.invoke(
 {
    "context":"My name is Aaron.",
    "question":"What's my name ?"
 })

'Of course! Based on the context provided, your name is Aaron.'

In [27]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

ImportError: Could not import docarray python package. Please install it with `pip install docarray`.